# Software architecture for Reachy

In [ ]:
from reachy import Reachy

reachy = Reachy()

reachy.compliant = False

for m in reachy.motors:
    m.goal_position = 0

* Keep it simple.
* Fast protoyping.
* Efficient.

# Overview - Three main layers

* [pypot](https://github.com/poppy-project/pypot): low-level communication with dynamixel motors and few basic sensors

* [reachy](https://github.com/pollen-robotics/reachy/tree/master/software): specific robot (or creature) configuration on top 

* extensions and plugins: IK, VREP, motion, demonstration, etc.

## Mostly written in Python

* Supported versions: >= 2.7 or >= 3.4

* Works on Win, Mac, Linux

#### Why?

* fast prototyping
* compatibility with scientific libraries (numpy, scipy, matplotlib, opencv...)

# APIs

Can be easily connected to other languages / interfaces

* Rest API
* ZMQ Client/Server

# Open Source


* pypot: GPLv3
* reachy: LGPL

## Community

* about 25 contributors, 85 forks, about 300 users a month
* used on few different robots

# Use of Jupyter Notebook

![Jupyter Notebook](http://jupyter.org/assets/jupyterpreview.png)

# Use of Jupyter Notebook

* Great for experimenting, documenting.
* Most of the doc / examples / even this presentation.
* The different examples we will try this week.

# Pypot

## Installation

From Pypi
* pip install pypot

From the source
* pip install -e .

# Current version

In [1]:
import pypot

pypot.__version__

'3.1.1'

# Low-level API for dynamixel motor control

In [ ]:
from pypot.dynamixel import DxlIO

# Connect to the motor bus via serial 
# Reachy motors are configured in 1M bauds.
io = DxlIO('/dev/tty.usbmodem14341', baudrate=1000000)

# Scan available motors

In [ ]:
ids = io.scan()
print(ids)

# Read register values 

In [ ]:
io.get_present_position(ids)

In [ ]:
io.get_present_temperature(ids)

In [ ]:
# many others, can be found using introspection, completion
# Or in the doc: http://poppy-project.github.io/pypot/api.html

io.get_

# Write register values

In [ ]:
io.enable_torque(ids)

In [ ]:
pos = {
    # id: pos
    24: 0,
    25: 0
}

io.set_goal_position(pos)

# Clean and close

In [ ]:
io.disable_torque(ids)

io.close()

# Low-level

Fast but not so easy to write.
Generic. Not robot specific.

Probably not what you want/need to use. 

Still really useful to understand how it works!

# Reachy - a robot creature

## Install from source

```bash
git clone https://github.com/pollen-robotics/reachy.git
pip install -e ./reachy/software
```

# Reachy

In [ ]:
from reachy import Leachy

leachy = Leachy(brunel_hand='/dev/tty.usbmodem14141')

Handles all low-level connection and communication.

# Robot specific configuration

In [ ]:
print(leachy.motors)

In [ ]:
print(leachy.l_shoulder_roll)

# How does it work?

Simple JSON [configuration](https://github.com/pollen-robotics/reachy/blob/master/software/reachy/configuration/reachy.json) file that describes the robot.

# Reading values

In [ ]:
print(leachy.l_arm_yaw.present_position)

In [ ]:
print(leachy.l_shoulder_roll.present_temperature)

Asynchronous! (refresh ~50Hz)

# Writing values

In [ ]:
leachy.l_arm_yaw.compliant = False

In [ ]:
leachy.l_arm_yaw.goal_position = 0

In [ ]:
leachy.l_arm_yaw.goal_position = 20

Also asynchronous!

In [ ]:
# Will most likely stay in 20.
leachy.l_arm_yaw.goal_position = 0
leachy.l_arm_yaw.goal_position = 20

# Reachy's API

Let you write more simple code.

In [ ]:
for m in leachy.motors:
    m.compliant = False
    m.goto_position(0, 2)

In [ ]:
import time

leachy.l_arm_yaw.moving_speed = 50
leachy.l_forearm_yaw.compliant = True

while True:
    leachy.l_arm_yaw.goal_position = -leachy.l_forearm_yaw.present_position
    time.sleep(1/50)

# Hand support

Directly accessible via ```reachy.hand```.

In [ ]:
leachy.hand.close()

In [ ]:
leachy.hand.open()

In [ ]:
while True:
    if abs(leachy.l_shoulder_pitch.present_position) > 20:
        leachy.hand.close()
    else:
        leachy.hand.open()

Only a sneak peek. More details and examples this week...

# Extensions and third parties tools

# Run in simulation

![Reachy in VREP](./img/reachy-vrep.png)

# Reachy in VREP

## Connect to the "real" robot

In [ ]:
reachy = Reachy()

## Connect to a simulated robot

In [ ]:
# First, launch VREP (version 3.2)
# Connect to a "simulated" robot
reachy = Reachy(simulator='vrep')

Then, use the same APIs (as much as possible).

# IKPY

Forward and inverse kinematics.

Uses the [URDF](https://github.com/pollen-robotics/reachy/tree/master/hardware/URDF) file.

# Forward kinematics

Gives you the 3D cartesian position of the end effector.

In [ ]:
print(reachy.ik_chain.end_effector)

# Inverse kinematics

Computes joints position to reach a 3D cartesian position.

In [ ]:
reachy.ik_chain.goto(x, y, z)

* This is a hard problem, with multiple solutions!

* With only a theoretical model of the robot.

# Primitives motion

In [ ]:
from pypot.primitive.utils import Sinus

s = Sinus(reachy, 50, [reachy.r_forearm_yaw], amp=20, freq=1)

In [ ]:
s.start()

In [ ]:
s.pause()

In [ ]:
s.resume()

In [ ]:
s.stop()

Can be combined to create more complex motions.

# Motion demonstration

In [ ]:
from pypot.primitive.move import MoveRecorder

In [ ]:
record = MoveRecorder(reachy, 50, reachy.motors)

In [ ]:
record.start()

In [ ]:
record.stop()

# Plot move

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(record.move)

# Re-play move

In [ ]:
from pypot.primitive.move import MovePlayer

player = MovePlayer(reachy, record.move)

In [ ]:
player.start()

# Going further

* Machine learning
* Vision
* Complex demonstration
* Motor control
* Safety
* ...

# Thanks for your attention!

# Questions?

https://vimeo.com/95405850